In [172]:
import pandas as pd
import numpy as np
import os

In [173]:
from google.colab import drive
drive.mount("/content/gdriv")

Drive already mounted at /content/gdriv; to attempt to forcibly remount, call drive.mount("/content/gdriv", force_remount=True).


In [174]:
data_path = "/content/gdriv/MyDrive/산데과"
data_list = os.listdir(data_path)
train_dt = pd.read_csv(data_path + "/" + data_list[7])
test_dt = pd.read_csv(data_path + "/" + data_list[8])

In [175]:
train_dt.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,male,22.0,1,0,7.2500,S,False
1,1,1,female,38.0,1,0,71.2833,C,False
2,1,3,female,26.0,0,0,7.9250,S,True
3,1,1,female,35.0,1,0,53.1000,S,False
4,0,3,male,35.0,0,0,8.0500,S,True


In [176]:
total_dt = pd.concat([train_dt,test_dt],axis=0)

In [177]:
total_dt

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0.0,3,male,22.0,1,0,7.2500,S,False
1,1.0,1,female,38.0,1,0,71.2833,C,False
2,1.0,3,female,26.0,0,0,7.9250,S,True
3,1.0,1,female,35.0,1,0,53.1000,S,False
4,0.0,3,male,35.0,0,0,8.0500,S,True
...,...,...,...,...,...,...,...,...,...
45,NaN,3,female,39.0,0,5,29.1250,Q,False
46,NaN,2,male,27.0,0,0,13.0000,S,True
47,NaN,1,female,19.0,0,0,30.0000,S,True
48,NaN,1,male,26.0,0,0,30.0000,C,True


In [178]:
age_index=[]
for i in total_dt['age']:
  if i<10:
    age_index.append(1)
  elif 10<=i and i<20:
    age_index.append(1)
  elif 20<=i and i<30:
    age_index.append(2)
  elif 30<=i and i<40:
    age_index.append(2)
  elif 40<=i and i<50:
    age_index.append(3)
  elif 50<=i and i<60:
    age_index.append(3)
  else:
    age_index.append(3)

pd.Series(age_index).value_counts()
total_dt['age']=age_index

In [179]:
pd.Series(age_index).value_counts()

2    386
3    162
1    157
dtype: int64

In [180]:
fare_index = []
for i in total_dt['fare']:
  if i<10:
    fare_index.append(1)
  elif 10<=i and i<30:
    fare_index.append(2)
  else:
    fare_index.append(3)

pd.Series(fare_index).value_counts()
total_dt['fare']=fare_index

In [181]:
total_dt = pd.get_dummies(total_dt,prefix_sep='_',drop_first=True)

In [182]:
total_dt.head()

,survived,pclass,age,sibsp,parch,fare,alone,sex_male,embarked_Q,embarked_S
0,0.0,3,2,1,0,1,False,1,0,1
1,1.0,1,2,1,0,3,False,0,0,0
2,1.0,3,2,0,0,1,True,0,0,1
3,1.0,1,2,1,0,3,False,0,0,1
4,0.0,3,2,0,0,1,True,1,0,1


In [183]:
test=total_dt[total_dt['survived'].isna()]
train=total_dt[total_dt['survived'].notna()]

In [184]:
test.head()

,survived,pclass,age,sibsp,parch,fare,alone,sex_male,embarked_Q,embarked_S
0,NaN,3,1,1,0,2,False,0,0,0
1,NaN,3,2,0,0,1,True,1,0,1
2,NaN,3,1,0,0,1,True,1,0,1
3,NaN,1,2,1,1,3,False,0,0,0
4,NaN,3,2,0,0,1,True,1,0,1


In [185]:
train.head()

,survived,pclass,age,sibsp,parch,fare,alone,sex_male,embarked_Q,embarked_S
0,0.0,3,2,1,0,1,False,1,0,1
1,1.0,1,2,1,0,3,False,0,0,0
2,1.0,3,2,0,0,1,True,0,0,1
3,1.0,1,2,1,0,3,False,0,0,1
4,0.0,3,2,0,0,1,True,1,0,1


In [186]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix

In [187]:
y = train.iloc[:,0]
x = train.iloc[:,1:16]

In [188]:
 trainx, testx, trainy, testy = train_test_split(x, y, test_size=0.3, random_state=0)

In [189]:
model = CategoricalNB()

In [190]:
model.fit(trainx,trainy)

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [191]:
train_pred=model.predict(trainx)

In [192]:
m = confusion_matrix(trainy, train_pred)
print(m)
np.sum(np.diag(m))/len(trainx)

[[214  58]
 [ 53 133]]


0.75764192139738

In [193]:
train_pred=model.predict(x)
m = confusion_matrix(y, train_pred)
print(m)
np.sum(np.diag(m))/len(x)

[[314  80]
 [ 78 183]]


0.7587786259541984

In [194]:
test_pred = model.predict(testx)
m = confusion_matrix(testy, test_pred)
np.sum(np.diag(m))/len(testy)

0.7614213197969543

In [197]:
model = CategoricalNB()
model.fit(x,y)
test_pred = model.predict(test.iloc[:,1:16])

In [198]:
test_pred = pd.DataFrame(test_pred)

In [199]:
test_pred.to_csv("/content/gdriv/MyDrive/산데과/201711514 박주호 과제8.csv")